<a href="https://colab.research.google.com/github/IsaGOLisboa/Empresas_B3_variaveis/blob/main/Obten%C3%A7%C3%A3o_de_Dados_CVM_B3_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bibliotecas

# Nova seção

Instalando Bibliotecas

In [ ]:
!pip install python-docx
!pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 710.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53895 sha256=db25fa0c727f169e35defd6e554d439bc3bcb5fc8b11c18405306dcaed4cfc8a
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [ ]:
import requests
import zipfile
import os
import pandas as pd
from google.colab.data_table import DataTable
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display
from docx import Document
from tqdm import tqdm
from datetime import datetime
import shutil

## **Extrair os arquivos da CVM**

Diretórios necessários para salvar/obter arquivos que serão gerados/utilizados no código

In [ ]:
#Arquivos Históricos
# Diretório de destino para salvar os arquivos ZIP que serão exrtraídos da URL
diretorio_destino_arq_zip = '/content/drive/MyDrive/Michel_CVM_B3/arquivos_cvm_zip'

# Diretório para salvar os arquivos descompactados (após o download é necessário descompactar por estarem no formato .zip)
diretorio_destino_arq_csv = '/content/drive/MyDrive/Michel_CVM_B3/descompactados'

#Arquivos Trimestrais (local onde serão salvos os arquivos em formato .zip e .csv após descompactar)
diretorio_destino_arq_trim = '/content/drive/MyDrive/Michel_CVM_B3/relatorios_trimestrais'

#diretório para salvar as listas para CNPJ (pasta anexa para salvar a lista obtida com os nomes das empresas e respectivos cnpjs obtidos)
diretório_destino_lista = '/content/drive/MyDrive/Michel_CVM_B3/listas_cnpj'

# Caminho para os arquivos DOCX (local onde foram salvos os arquivos .docx criados para guardar os elementos extraídos da B3 com os nomes das empresas dispensadas da CVM)
caminho_arq_docx_emp_dispensadas = '/content/drive/MyDrive/arquivos_cvm_zip/Outros arquivos/Empresas_dispensadas_CVM/'

# Caminho para os arquivos DOCX (local onde foram salvos os arquivos .docx criados para guardar os elementos extraídos da B3 com os nomes das empresas não dispensadas e local para salvar a lista das empresas após extração do HTML)
caminho_arq_docx_emp_list_b3 = Document("/content/drive/MyDrive/arquivos_cvm_zip/Outros arquivos/elemento_html_b3.docx")
arq_tentativa_extracao_url = '/content/drive/MyDrive/arquivos_cvm_zip/Outros arquivos/empresas_listadas_b3.csv'

#Caminho para salvar os relatórios de indicadores financeiros
diretorio_rel_ind_financeiros = '/content/drive/MyDrive/Michel_CVM_B3/rel_indicadores_financeiros'

In [ ]:
# Anos em que os relatórios serão obtidos
ano_inicial = 2012
ano_final = 2022

In [ ]:
url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS'

# Diretório de destino para salvar os arquivos ZIP
diretorio_destino = diretorio_destino_arq_zip

# Diretório para salvar os arquivos descompactados
diretorio_descompactados = diretorio_destino_arq_csv

# Verificando se o diretório de destino existe
os.makedirs(diretorio_destino, exist_ok=True)

for ano in range(ano_inicial, ano_final + 1):
    relatorio = "dfp_cia_aberta"
    arquivo_zip = f'{relatorio}_{ano}.zip'
    caminho_arquivo_local = os.path.join(diretorio_destino, arquivo_zip)

    # Verificando se o arquivo já existe no diretório de destino
    if os.path.exists(caminho_arquivo_local):
        print(f"Arquivo {arquivo_zip} já existe. Pulando o download e extração.")
    else:
        url_arquivo = f'{url_base}/{arquivo_zip}'

        # Download do arquivo ZIP
        response = requests.get(url_arquivo)
        if response.status_code == 200:
            with open(caminho_arquivo_local, 'wb') as arquivo_local:
                arquivo_local.write(response.content)

            # Extração do conteúdo do arquivo ZIP
            with zipfile.ZipFile(caminho_arquivo_local, 'r') as zip_ref:
                # Coletar os nomes dos arquivos extraídos
                lista_arquivos_zip = zip_ref.namelist()

                # Verifica se os arquivos CSV já existem na pasta do ano
                csv_faltantes = [f'{relatorio}_{ano}.csv' for arquivo in lista_arquivos_zip]
                csv_faltantes = [csv for csv in csv_faltantes if not os.path.exists(os.path.join(diretorio_descompactados, str(ano), csv))]

                if csv_faltantes:
                    # Cria um diretório para cada ano
                    diretorio_ano = os.path.join(diretorio_descompactados, str(ano))
                    os.makedirs(diretorio_ano, exist_ok=True)

                    # Exibindo a lista de nomes dos arquivos extraídos
                    print("Arquivos no arquivo ZIP:")
                    for arquivo in lista_arquivos_zip:
                        print(arquivo)

                        # Determinando qual arquivo do ZIP será lido.
                        with zip_ref.open(arquivo) as arquivo_zip:
                            df = pd.read_csv(arquivo_zip, encoding='cp1252', sep=';')

                            # Salva o DataFrame como um arquivo CSV na pasta do ano correspondente
                            caminho_arquivo_dataframe = os.path.join(diretorio_ano, f'{arquivo[:-4]}.csv')
                            df.to_csv(caminho_arquivo_dataframe, index=False)
                            print(f"DataFrame salvo em: {caminho_arquivo_dataframe}")
                else:
                    print(f"Arquivos CSV já existem na pasta {diretorio_ano}. Pulando a extração.")
        else:
            print(f"Falha ao baixar {arquivo_zip}. Código de status:", response.status_code)

Arquivo dfp_cia_aberta_2012.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2013.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2014.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2015.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2016.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2017.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2018.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2019.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2020.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2021.zip já existe. Pulando o download e extração.
Arquivo dfp_cia_aberta_2022.zip já existe. Pulando o download e extração.


Relatório trimestral ano corrente

In [ ]:
ano_corrente = ano_final + 1

file_url = f'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_{ano_corrente}.zip'
save_folder = diretorio_destino_arq_trim
zip_file_path = os.path.join(save_folder, f'itr_cia_aberta_{ano_corrente}.zip')
extracted_folder_path = os.path.join(save_folder, f'itr_cia_aberta_{ano_corrente}')

# Verifica se o arquivo zip já existe no diretório
if not os.path.exists(zip_file_path):
    # Faz o download do arquivo ZIP caso não exista
    response = requests.get(file_url)
    if response.status_code == 200:
        with open(zip_file_path, 'wb') as f:
            f.write(response.content)
        print("Download do arquivo ZIP concluído com sucesso!")
    else:
        print("Falha ao baixar o arquivo ZIP.")
else:
    print("Arquivo ZIP já existe no diretório. Pulando o download.")

# Verifica se o arquivo zip existe no diretório
if os.path.exists(zip_file_path):
    # Extrai todos os arquivos do ZIP para um diretório temporário
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(save_folder)

    # Verifica e atualiza os arquivos se houver modificações
    if os.path.exists(extracted_folder_path):
        for root, dirs, files in os.walk(extracted_folder_path):
            for file in files:
                extracted_file_path = os.path.join(root, file)
                target_file_path = os.path.join(save_folder, file)
                if not os.path.exists(target_file_path):
                    shutil.move(extracted_file_path, save_folder)
                    print(f"O arquivo '{file}' foi adicionado.")
                else:
                    zip_file_mod_time = datetime.fromtimestamp(os.path.getmtime(extracted_file_path))
                    target_file_mod_time = datetime.fromtimestamp(os.path.getmtime(target_file_path))
                    if zip_file_mod_time > target_file_mod_time:
                        os.remove(target_file_path)
                        shutil.move(extracted_file_path, save_folder)
                        print(f"O arquivo '{file}' foi atualizado.")
                    else:
                        os.remove(extracted_file_path)

    # Remove o diretório temporário extraído do ZIP
    shutil.rmtree(extracted_folder_path, ignore_errors=True)
    print("Verificação e atualização concluídas.")
else:
    print("Arquivo ZIP não encontrado no diretório.")


Arquivo ZIP já existe no diretório. Pulando o download.
Verificação e atualização concluídas.


Visualizando os arquivos obtidos

In [ ]:
#Lista com todos os arquivos no diretório disponíveis para visualização
ano = ano_final
arquivos_disponiveis = os.listdir(os.path.join(diretorio_descompactados, str(ano)))

# Exibe a lista de arquivos disponíveis
for arquivo in arquivos_disponiveis:
    print(arquivo)

dfp_cia_aberta_2022.csv
dfp_cia_aberta_BPA_con_2022.csv
dfp_cia_aberta_BPA_ind_2022.csv
dfp_cia_aberta_BPP_con_2022.csv
dfp_cia_aberta_BPP_ind_2022.csv
dfp_cia_aberta_DFC_MD_con_2022.csv
dfp_cia_aberta_DFC_MD_ind_2022.csv
dfp_cia_aberta_DFC_MI_con_2022.csv
dfp_cia_aberta_DFC_MI_ind_2022.csv
dfp_cia_aberta_DMPL_con_2022.csv
dfp_cia_aberta_DMPL_ind_2022.csv
dfp_cia_aberta_DRA_con_2022.csv
dfp_cia_aberta_DRA_ind_2022.csv
dfp_cia_aberta_DRE_con_2022.csv
dfp_cia_aberta_DRE_ind_2022.csv
dfp_cia_aberta_DVA_con_2022.csv
dfp_cia_aberta_DVA_ind_2022.csv


In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DVA_con" #inserir apenas o nome do arquivo, sem o ano

# Definindo o caminho do arquivo usando formatação de string
caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')




===============================================================================

================================================================================

## **Obtenção das variáveis dentros dos arquivos**

Visualização

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DVA_con" #inserir apenas o nome do arquivo, sem o ano

# Definindo o caminho do arquivo usando formatação de string
caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

lista_ds_conta = df[['CD_CONTA', 'DS_CONTA']].drop_duplicates()

lista_ds_conta = lista_ds_conta.sort_values(by=['DS_CONTA'])

df_lista = pd.DataFrame(lista_ds_conta)

df_lista


,CD_CONTA,DS_CONTA
15802,7.01.02.01,( - ) Deduções de Vendas
22560,7.01.02.02,(-) Abatimentos e cancelamentos
17760,7.01.02.01,(-) Abatimentos e descontos
22474,7.01.02.01,(-) Deduções de Vendas
20676,7.04.02.01,(Constituição) / reversão parcial da perda est...
...,...,...
214,7.06,Vlr Adicionado Recebido em Transferência
5786,7.02.04.02,amortização mais valia de estoques
21152,7.08.03.03.02,"juros, variações cambiais e despesas com finan..."
21452,7.02.04.02,Água energia e gás


Receita Líquida

Histórico

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar a Receita Operacional Líquida
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.01')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
receita_operacional = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
receita_operacional.name = None
receita_liquida = receita_operacional



Trimestral

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar a Receita Operacional Líquida do Período
receita_liquida_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '3.01')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
receita_liquida_tri['DT_INI_EXERC'] = pd.to_datetime(receita_liquida_tri['DT_INI_EXERC'])
receita_liquida_tri['DT_FIM_EXERC'] = pd.to_datetime(receita_liquida_tri['DT_FIM_EXERC'])

# Encontrar os índices da maior 'DT_INI_EXERC' quando houver duas datas 'DT_FIM_EXERC' iguais
indices_max_DT_INI = receita_liquida_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DT_INI_EXERC'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_INI_EXERC'
receita_liquida_trime = receita_liquida_tri.loc[indices_max_DT_INI]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
receita_liquida_trime = receita_liquida_trime[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
receita_liquida_trime['Ano_tri'] = ano_corrente




<ipython-input-11-6ab837bd251b>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-11-6ab837bd251b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receita_liquida_tri['DT_INI_EXERC'] = pd.to_datetime(receita_liquida_tri['DT_INI_EXERC'])
<ipython-input-11-6ab837bd251b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
receita_liquida_trime['DT_REFER'] = pd.to_datetime(receita_liquida_trime['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
receita_liquida_trime['Quarter'] = receita_liquida_trime['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = receita_liquida_trime.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]
receita_liquida_trim = pivot


Ativo total (BPA)

Histórico

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPA_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpa = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPA = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '1')]

    # Selecionando colunas desejadas
    var_BPA = var_BPA[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPA['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpa = pd.concat([resultados_bpa, var_BPA], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpa = resultados_bpa.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
ativo_total = resultados_agregados_bpa.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
ativo_total.name = None


Trimestral

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPA_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Lucro Líquido do Período
var_BPA_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '1')]

# Converter a coluna 'DT_REFER' para o tipo datetime
var_BPA_tri['DT_REFER'] = pd.to_datetime(var_BPA_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = var_BPA_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
var_BPA_tri = var_BPA_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
var_BPA_tri = var_BPA_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
var_BPA_tri['Ano_tri'] = ano_corrente

ativo_total_tri = var_BPA_tri




<ipython-input-14-995cb6dddf9c>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-14-995cb6dddf9c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_BPA_tri['DT_REFER'] = pd.to_datetime(var_BPA_tri['DT_REFER'])


In [ ]:
ativo_total_tri

,DENOM_CIA,CNPJ_CIA,VL_CONTA,DT_REFER,Ano_tri
44229,2W ECOBANK S.A.,08.773.135/0001-00,2719101.0,2023-03-31,2023
44371,2W ECOBANK S.A.,08.773.135/0001-00,3069259.0,2023-06-30,2023
59839,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,7982238.0,2023-03-31,2023
59983,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,18679351.0,2023-06-30,2023
60121,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,19276056.0,2023-09-30,2023
...,...,...,...,...,...
51003,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,468737.0,2023-03-31,2023
51127,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,499737.0,2023-06-30,2023
63867,ZAMP SA,13.574.594/0001-96,4036969.0,2023-03-31,2023
63997,ZAMP SA,13.574.594/0001-96,4016531.0,2023-06-30,2023


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
ativo_total_tri['DT_REFER'] = pd.to_datetime(ativo_total_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
ativo_total_tri['Quarter'] = ativo_total_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = ativo_total_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

ativo_total_trim = pivot
ativo_total_trim


,DENOM_CIA,CNPJ_CIA,Q1,Q2,Q3
0,2W ECOBANK S.A.,08.773.135/0001-00,2719101.0,3.069259e+06,0.0
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,7982238.0,1.867935e+07,19276056.0
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,17873043.0,1.886233e+07,28584844.0
3,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,3288865.0,3.181564e+06,3821647.0
4,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,20021297.0,1.994339e+07,19825638.0
...,...,...,...,...,...
452,YBYRÁ CAPITAL S.A.,02.217.319/0001-07,2686639.0,4.665229e+09,0.0
453,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,9573709.0,9.722947e+06,0.0
454,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,1726189.0,1.774430e+06,0.0
455,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,468737.0,4.997370e+05,0.0


Patrimônio Líquido (BPP)

Histórico

In [ ]:
# Definindo relatório a ser aberto
relatorio = 'dfp_cia_aberta_BPP_con'

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpp = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPP = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '2.03')]

    # Selecionando colunas desejadas
    var_BPP = var_BPP[['DENOM_CIA','CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPP['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpp = pd.concat([resultados_bpp, var_BPP], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpp = resultados_bpp.groupby(['DENOM_CIA','CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
patrimonio_liquido_con = resultados_agregados_bpp.pivot_table(index=['DENOM_CIA','CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
patrimonio_liquido_con.name = None




In [ ]:
#Participações dos acionistas não controladores
# Definindo relatório a ser aberto
relatorio = 'dfp_cia_aberta_BPP_con'

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpp = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPP = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '2.03.09')]

    # Selecionando colunas desejadas
    var_BPP = var_BPP[['DENOM_CIA','CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPP['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpp = pd.concat([resultados_bpp, var_BPP], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpp = resultados_bpp.groupby(['DENOM_CIA','CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
part_ac_n_controladores = resultados_agregados_bpp.pivot_table(index=['DENOM_CIA','CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
part_ac_n_controladores.name = None


<ipython-input-97-57425cd60937>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_BPP['Ano'] = ano


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
patrimonio_liquido_con.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
part_ac_n_controladores.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
patrimonio_liquido_con.fillna(0, inplace=True)
part_ac_n_controladores.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
patrimonio_liquido = patrimonio_liquido_con.sub(part_ac_n_controladores, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
patrimonio_liquido.reset_index(inplace=True)


Trimestral

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Patrimônio Líquido Consolidado
var_BPP_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '2.03')]

# Converter a coluna 'DT_REFER' para o tipo datetime
var_BPP_tri['DT_REFER'] = pd.to_datetime(var_BPP_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = var_BPP_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
var_BPP_tri = var_BPP_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
var_BPP_tri = var_BPP_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
var_BPP_tri['Ano_tri'] = ano_corrente

patrimonio_liquido_con_tri = var_BPP_tri


<ipython-input-99-09ed6e891c87>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-99-09ed6e891c87>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_BPP_tri['DT_REFER'] = pd.to_datetime(var_BPP_tri['DT_REFER'])


In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Patrimônio Líquido Consolidado
var_BPP_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '2.03.09')]

# Converter a coluna 'DT_REFER' para o tipo datetime
var_BPP_tri['DT_REFER'] = pd.to_datetime(var_BPP_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = var_BPP_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
var_BPP_tri = var_BPP_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
var_BPP_tri = var_BPP_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
var_BPP_tri['Ano_tri'] = ano_corrente

part_ac_n_controladores_tri = var_BPP_tri



<ipython-input-100-0f7d6b98281d>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-100-0f7d6b98281d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_BPP_tri['DT_REFER'] = pd.to_datetime(var_BPP_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
patrimonio_liquido_con_tri['DT_REFER'] = pd.to_datetime(patrimonio_liquido_con_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
patrimonio_liquido_con_tri['Quarter'] = patrimonio_liquido_con_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = patrimonio_liquido_con_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

patrimonio_liquido_con_trim = pivot


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
part_ac_n_controladores_tri['DT_REFER'] = pd.to_datetime(part_ac_n_controladores_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
part_ac_n_controladores_tri['Quarter'] = part_ac_n_controladores_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = part_ac_n_controladores_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

part_ac_n_controladores_trim = pivot
part_ac_n_controladores_trim


,DENOM_CIA,CNPJ_CIA,Q1,Q2,Q3
0,2W ECOBANK S.A.,08.773.135/0001-00,-10.0,0.0,0.0
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,62248.0,64619.0,70655.0
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,478638.0,565665.0,4446401.0
3,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,0.0,0.0,0.0
4,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,1125909.0,1041935.0,1070177.0
...,...,...,...,...,...
433,YBYRÁ CAPITAL S.A.,02.217.319/0001-07,0.0,0.0,0.0
434,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,13873.0,13204.0,0.0
435,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,177927.0,175306.0,0.0
436,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,42044.0,37822.0,0.0


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
patrimonio_liquido_con_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
part_ac_n_controladores_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
patrimonio_liquido_con_trim.fillna(0, inplace=True)
part_ac_n_controladores_trim.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
patrimonio_liquido_trim = patrimonio_liquido_con_trim.sub(part_ac_n_controladores_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
patrimonio_liquido_trim.reset_index(inplace=True)

Lucro Líquido consolidado do período (DRE)

Histórico

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.11')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
lucro_prej_liquido = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
lucro_prej_liquido.columns.name = None







<ipython-input-104-c5f131149c92>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_DRE['Ano'] = ano


In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.11.02')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
atrib_n_socios_control = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
atrib_n_socios_control.columns.name = None



In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
lucro_prej_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
atrib_n_socios_control.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
lucro_prej_liquido.fillna(0, inplace=True)
atrib_n_socios_control.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
lucro_liquido = lucro_prej_liquido.sub(atrib_n_socios_control, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
lucro_liquido.reset_index(inplace=True)


Trimestral

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Lucro Líquido do Período
lucro_liquido_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '3.11')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
lucro_liquido_tri['DT_INI_EXERC'] = pd.to_datetime(lucro_liquido_tri['DT_INI_EXERC'])
lucro_liquido_tri['DT_FIM_EXERC'] = pd.to_datetime(lucro_liquido_tri['DT_FIM_EXERC'])

# Encontrar os índices da maior 'DT_INI_EXERC' quando houver duas datas 'DT_FIM_EXERC' iguais
indices_max_DT_INI = lucro_liquido_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DT_INI_EXERC'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_INI_EXERC'
lucro_liquido_con_tri = lucro_liquido_tri.loc[indices_max_DT_INI]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
lucro_liquido_con_tri = lucro_liquido_con_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
lucro_liquido_con_tri['Ano_tri'] = ano_corrente




<ipython-input-107-74fdc9998d62>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-107-74fdc9998d62>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lucro_liquido_tri['DT_INI_EXERC'] = pd.to_datetime(lucro_liquido_tri['DT_INI_EXERC'])
<ipython-input-107-74fdc9998d62>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
lucro_liquido_con_tri['DT_REFER'] = pd.to_datetime(lucro_liquido_con_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
lucro_liquido_con_tri['Quarter'] = lucro_liquido_con_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = lucro_liquido_con_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='last')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

lucro_liquido_con_trim = pivot


In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Lucro Líquido do Período
atrib_n_socios_control_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '3.11.02')]

# Converter a coluna 'DT_REFER' para o tipo datetime
atrib_n_socios_control_tri['DT_REFER'] = pd.to_datetime(atrib_n_socios_control_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = atrib_n_socios_control_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
atrib_n_socios_control_tri = atrib_n_socios_control_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
atrib_n_socios_control_tri = atrib_n_socios_control_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
atrib_n_socios_control_tri['Ano_tri'] = ano_corrente


<ipython-input-109-359efc556202>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-109-359efc556202>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atrib_n_socios_control_tri['DT_REFER'] = pd.to_datetime(atrib_n_socios_control_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
atrib_n_socios_control_tri['DT_REFER'] = pd.to_datetime(atrib_n_socios_control_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
atrib_n_socios_control_tri['Quarter'] = atrib_n_socios_control_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = atrib_n_socios_control_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

atrib_n_socios_control_trim = pivot  # Renomeando o DataFrame final para atrib_n_socios_control_trim


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
lucro_liquido_con_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
atrib_n_socios_control_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
lucro_liquido_con_trim.fillna(0, inplace=True)
atrib_n_socios_control_trim.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
lucro_liquido_trim = lucro_liquido_con_trim.sub(atrib_n_socios_control_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
lucro_liquido_trim.reset_index(inplace=True)

Dívida líquida = empréstimos e financiamentos - caixa equivalente de caixa

Histórico

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPP_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpp = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPP = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '2.01.04')]

    # Selecionando colunas desejadas
    var_BPP = var_BPP[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPP['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpp = pd.concat([resultados_bpp, var_BPP], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpp = resultados_bpp.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
divida_total_1 = resultados_agregados_bpp.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='first').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
divida_total_1.name = None


In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPP_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpp = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPP = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '2.02.01')]

    # Selecionando colunas desejadas
    var_BPP = var_BPP[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPP['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpp = pd.concat([resultados_bpp, var_BPP], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpp = resultados_bpp.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
divida_total_2 = resultados_agregados_bpp.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='first').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
divida_total_2.name = None

In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
divida_total_1.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
divida_total_2.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
divida_total_1.fillna(0, inplace=True)
divida_total_2.fillna(0, inplace=True)

# Somando os valores para obter divida_liquida
divida_total = divida_total_1.add(divida_total_2, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
divida_total.reset_index(inplace=True)

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPA_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpa = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPA = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '1.01.01')]

    # Selecionando colunas desejadas
    var_BPA = var_BPA[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPA['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpa = pd.concat([resultados_bpa, var_BPA], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpa= resultados_bpa.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
cx_equiv_cx = resultados_agregados_bpa.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='first').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
cx_equiv_cx.name = None


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
divida_total.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
cx_equiv_cx.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
divida_total.fillna(0, inplace=True)
cx_equiv_cx.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
divida_liquida = divida_total.sub(cx_equiv_cx, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
divida_liquida.reset_index(inplace=True)


Trimestral

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar a Dívida Total
divida_total_1_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '2.01.04')]

# Converter a coluna 'DT_REFER' para o tipo datetime
divida_total_1_tri['DT_REFER'] = pd.to_datetime(divida_total_1_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = divida_total_1_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
divida_total_1_tri = divida_total_1_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
divida_total_1_tri = divida_total_1_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
divida_total_1_tri['Ano_tri'] = ano_corrente



<ipython-input-117-5b3f36ad913f>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-117-5b3f36ad913f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  divida_total_1_tri['DT_REFER'] = pd.to_datetime(divida_total_1_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
divida_total_1_tri['DT_REFER'] = pd.to_datetime(divida_total_1_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
divida_total_1_tri['Quarter'] = divida_total_1_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = divida_total_1_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='last')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc. sem o ano presente
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

divida_total_1_trim = pivot


In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar a Dívida Total
divida_total_2_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '2.02.01')]

# Converter a coluna 'DT_REFER' para o tipo datetime
divida_total_2_tri['DT_REFER'] = pd.to_datetime(divida_total_2_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = divida_total_2_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
divida_total_2_tri = divida_total_2_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
divida_total_2_tri = divida_total_2_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
divida_total_2_tri['Ano_tri'] = ano_corrente


<ipython-input-119-2a8aa2af7247>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-119-2a8aa2af7247>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  divida_total_2_tri['DT_REFER'] = pd.to_datetime(divida_total_2_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
divida_total_2_tri['DT_REFER'] = pd.to_datetime(divida_total_2_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
divida_total_2_tri['Quarter'] = divida_total_2_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = divida_total_2_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='last')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

divida_total_2_trim = pivot


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
divida_total_1_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
divida_total_2_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
divida_total_1_trim.fillna(0, inplace=True)
divida_total_2_trim.fillna(0, inplace=True)

# Somando os valores para obter divida_liquida
divida_total_trim = divida_total_1_trim.add(divida_total_2_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
divida_total_trim.reset_index(inplace=True)

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPA_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Saldo Final de Caixa e Equivalentes
cx_equiv_cx_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '1.01.01')]

# Converter a coluna 'DT_REFER' para o tipo datetime
cx_equiv_cx_tri['DT_REFER'] = pd.to_datetime(cx_equiv_cx_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = cx_equiv_cx_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
cx_equiv_cx_tri = cx_equiv_cx_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
cx_equiv_cx_tri = cx_equiv_cx_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
cx_equiv_cx_tri['Ano_tri'] = ano_corrente


<ipython-input-122-0ffeda38ac1e>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-122-0ffeda38ac1e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cx_equiv_cx_tri['DT_REFER'] = pd.to_datetime(cx_equiv_cx_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
cx_equiv_cx_tri['DT_REFER'] = pd.to_datetime(cx_equiv_cx_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
cx_equiv_cx_tri['Quarter'] = cx_equiv_cx_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = cx_equiv_cx_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='last')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

cx_equiv_cx_trim = pivot


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
divida_total_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
cx_equiv_cx_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
divida_total_trim.fillna(0, inplace=True)
cx_equiv_cx_trim.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida_tri
divida_liquida_trim = divida_total_trim.sub(cx_equiv_cx_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
divida_liquida_trim.reset_index(inplace=True)
# Removendo duplicatas baseadas na coluna 'DT_REFER'
divida_liquida_trim.drop_duplicates(subset=['DENOM_CIA', 'CNPJ_CIA'], inplace=True)


Ebitida

ebitida = LAIR + juros + depreciação + amortização

Histórico

Depreciação e amortização

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DVA_con"  # inserir apenas o nome do arquivo, sem o ano

# DataFrame vazio para armazenar os resultados
resultados_dva = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DVA = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '7.04.01')]

    # Selecionando colunas desejadas
    var_DVA = var_DVA[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DVA['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dva = pd.concat([resultados_dva, var_DVA], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dva = resultados_dva.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
deprec_amortz = resultados_agregados_dva.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
deprec_amortz.name = None


LAIR

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.05')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
LAIR = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
LAIR.name = None




In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
deprec_amortz.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
LAIR.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)


# Substituindo valores NaN por 0
deprec_amortz.fillna(0, inplace=True)
LAIR.fillna(0, inplace=True)

# Calculando o EBTIDA (soma das colunas deprec_amortz, juros e LAIR)
EBTIDA = LAIR.sub(deprec_amortz, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
EBTIDA.reset_index(inplace=True)

Trimestral

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DVA_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Lucro Líquido do Período
deprec_amortz_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '7.04.01')]

# Converter a coluna 'DT_REFER' para o tipo datetime
deprec_amortz_tri['DT_REFER'] = pd.to_datetime(deprec_amortz_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = deprec_amortz_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
deprec_amortz_tri = deprec_amortz_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
deprec_amortz_tri = deprec_amortz_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
deprec_amortz_tri['Ano_tri'] = ano_corrente




<ipython-input-128-1594f3be8d26>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-128-1594f3be8d26>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deprec_amortz_tri['DT_REFER'] = pd.to_datetime(deprec_amortz_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
deprec_amortz_tri['DT_REFER'] = pd.to_datetime(deprec_amortz_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
deprec_amortz_tri['Quarter'] = deprec_amortz_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = deprec_amortz_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Definir os trimestres Q2 e Q3 que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 estiver presente
if 'Q2' in trimestres_presentes:
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q2 and coluna_q1:
        pivot[coluna_q2[0]] = pivot[coluna_q2[0]] - pivot[coluna_q1[0]]

# Verificar e calcular Q3 - Q2 - Q1 se Q3 estiver presente
if 'Q3' in trimestres_presentes:
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q3 and coluna_q2 and coluna_q1:
        pivot[coluna_q3[0]] = pivot[coluna_q3[0]] - pivot[coluna_q2[0]] - pivot[coluna_q1[0]]

deprec_amortz_trim = pivot


In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Lucro Líquido do Período
LAIR_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '3.05')]

# Converter a coluna 'DT_REFER' para o tipo datetime
LAIR_tri['DT_REFER'] = pd.to_datetime(LAIR_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = LAIR_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
LAIR_tri = LAIR_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
LAIR_tri = LAIR_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
LAIR_tri['Ano_tri'] = ano_corrente


<ipython-input-130-837af74a7b96>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-130-837af74a7b96>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LAIR_tri['DT_REFER'] = pd.to_datetime(LAIR_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
LAIR_tri['DT_REFER'] = pd.to_datetime(LAIR_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
LAIR_tri['Quarter'] = LAIR_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = LAIR_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Definir os trimestres Q2 e Q3 que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 estiver presente
if 'Q2' in trimestres_presentes:
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q2 and coluna_q1:
        pivot[coluna_q2[0]] = pivot[coluna_q2[0]] - pivot[coluna_q1[0]]

# Verificar e calcular Q3 - Q2 - Q1 se Q3 estiver presente
if 'Q3' in trimestres_presentes:
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q3 and coluna_q2 and coluna_q1:
        pivot[coluna_q3[0]] = pivot[coluna_q3[0]] - pivot[coluna_q2[0]] - pivot[coluna_q1[0]]


LAIR_trim = pivot


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
deprec_amortz_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
LAIR_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
deprec_amortz_trim.fillna(0, inplace=True)
LAIR_trim.fillna(0, inplace=True)

# Calculando o EBTIDA (soma das colunas deprec_amortz_tri e LAIR_tri)
EBTIDA_trim = LAIR_trim.sub(deprec_amortz_trim, fill_value=0)


# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
EBTIDA_trim.reset_index(inplace=True)


Free cash flow = fluxo de caixa de atividades operacionais - investimentos em ativos de capital

Histórico

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DFC_MI_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_cfc_mi = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_CFC_MI = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '6.01')]

    # Selecionando colunas desejadas
    var_CFC_MI = var_CFC_MI[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_CFC_MI['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_cfc_mi = pd.concat([resultados_cfc_mi, var_CFC_MI], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_cfc_mi= resultados_cfc_mi.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
fluxo_caixa = resultados_agregados_cfc_mi.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
fluxo_caixa.name = None


In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DFC_MI_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_cfc_mi = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_CFC_MI = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '6.02')]

    # Selecionando colunas desejadas
    var_CFC_MI = var_CFC_MI[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_CFC_MI['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_cfc_mi = pd.concat([resultados_cfc_mi, var_CFC_MI], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_cfc_mi= resultados_cfc_mi.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
capex = resultados_agregados_cfc_mi.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
capex.name = None


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
fluxo_caixa.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
capex.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
fluxo_caixa.fillna(0, inplace=True)
capex.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
free_cash_flow = fluxo_caixa.add(capex, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
free_cash_flow.reset_index(inplace=True)

Trimestral

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Lucro Líquido do Período
fluxo_caixa_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '6.01')]

# Converter a coluna 'DT_REFER' para o tipo datetime
fluxo_caixa_tri['DT_REFER'] = pd.to_datetime(fluxo_caixa_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = fluxo_caixa_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
fluxo_caixa_tri = fluxo_caixa_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
fluxo_caixa_tri = fluxo_caixa_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
fluxo_caixa_tri['Ano_tri'] = ano_corrente



<ipython-input-136-6310829f68d0>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-136-6310829f68d0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluxo_caixa_tri['DT_REFER'] = pd.to_datetime(fluxo_caixa_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
fluxo_caixa_tri['DT_REFER'] = pd.to_datetime(fluxo_caixa_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
fluxo_caixa_tri['Quarter'] = fluxo_caixa_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = fluxo_caixa_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Definir os trimestres Q2 e Q3 que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 estiver presente
if 'Q2' in trimestres_presentes:
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q2 and coluna_q1:
        pivot[coluna_q2[0]] = pivot[coluna_q2[0]] - pivot[coluna_q1[0]]

# Verificar e calcular Q3 - Q2 - Q1 se Q3 estiver presente
if 'Q3' in trimestres_presentes:
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q3 and coluna_q2 and coluna_q1:
        pivot[coluna_q3[0]] = pivot[coluna_q3[0]] - pivot[coluna_q2[0]] - pivot[coluna_q1[0]]


fluxo_caixa_trim = pivot


In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Lucro Líquido do Período
capex_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '6.02')]

# Converter a coluna 'DT_REFER' para o tipo datetime
capex_tri['DT_REFER'] = pd.to_datetime(capex_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = capex_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
capex_tri = capex_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
capex_tri = capex_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
capex_tri['Ano_tri'] = ano_corrente


<ipython-input-138-80d82e45eadf>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-138-80d82e45eadf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capex_tri['DT_REFER'] = pd.to_datetime(capex_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
capex_tri['DT_REFER'] = pd.to_datetime(capex_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
capex_tri['Quarter'] = capex_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = capex_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Definir os trimestres Q2 e Q3 que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 estiver presente
if 'Q2' in trimestres_presentes:
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q2 and coluna_q1:
        pivot[coluna_q2[0]] = pivot[coluna_q2[0]] - pivot[coluna_q1[0]]

# Verificar e calcular Q3 - Q2 - Q1 se Q3 estiver presente
if 'Q3' in trimestres_presentes:
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q3 and coluna_q2 and coluna_q1:
        pivot[coluna_q3[0]] = pivot[coluna_q3[0]] - pivot[coluna_q2[0]] - pivot[coluna_q1[0]]


capex_trim = pivot


In [ ]:
capex_trim

,DENOM_CIA,CNPJ_CIA,Q1,Q2,Q3
0,2W ECOBANK S.A.,08.773.135/0001-00,-40014.0,-428805.0,468819.0
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,-138359.0,-7981002.0,-358088.0
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,-634594.0,-538237.0,-581441.0
3,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,-28245.0,-32099.0,-5013.0
4,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,-970424.0,-728622.0,-913467.0
...,...,...,...,...,...
442,YBYRÁ CAPITAL S.A.,02.217.319/0001-07,0.0,0.0,0.0
443,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,-225484.0,35129.0,190355.0
444,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,-6182.0,-9451.0,15633.0
445,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,2786.0,203.0,-2989.0


In [ ]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
fluxo_caixa_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'],  inplace=True)
capex_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
fluxo_caixa_trim.fillna(0, inplace=True)
capex_trim.fillna(0, inplace=True)

# Subtraindo os valores para obter free_cash_flow_tri
free_cash_flow_trim = fluxo_caixa_trim.add(capex_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
free_cash_flow_trim.reset_index(inplace=True)

In [ ]:
free_cash_flow_trim

,DENOM_CIA,CNPJ_CIA,Q1,Q2,Q3
0,2W ECOBANK S.A.,08.773.135/0001-00,-93378.0,-2.633300e+05,3.567080e+05
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,-90246.0,-8.083706e+06,9.366800e+04
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,-347039.0,-5.600850e+05,-3.469340e+05
3,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,-215027.0,8.740400e+04,-1.806950e+05
4,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,-924251.0,5.701220e+05,1.500290e+05
...,...,...,...,...,...
442,YBYRÁ CAPITAL S.A.,02.217.319/0001-07,-2129812.0,-4.662405e+09,4.664535e+09
443,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,77358.0,2.692470e+05,-3.466050e+05
444,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,-86213.0,-7.170200e+04,1.579150e+05
445,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,-5166.0,-1.269800e+04,1.786400e+04


Caixa Líquido atividades de finaciamento

Histórico

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DFC_MI_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_cfc_mi = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_CFC_MI = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '6.03')]

    # Selecionando colunas desejadas
    var_CFC_MI = var_CFC_MI[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_CFC_MI['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_cfc_mi = pd.concat([resultados_cfc_mi, var_CFC_MI], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_cfc_mi= resultados_cfc_mi.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
cx_liq_ativ_financ = resultados_agregados_cfc_mi.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
cx_liq_ativ_financ.name = None



Trimestral

In [ ]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)

# Filtrando os dados para encontrar o Lucro Líquido do Período
cx_liq_ativ_financ_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '6.03')]

# Converter a coluna 'DT_REFER' para o tipo datetime
cx_liq_ativ_financ_tri['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = cx_liq_ativ_financ_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
cx_liq_ativ_financ_tri = cx_liq_ativ_financ_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
cx_liq_ativ_financ_tri = cx_liq_ativ_financ_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
cx_liq_ativ_financ_tri['Ano_tri'] = ano_corrente



<ipython-input-144-fad40ba94602>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', error_bad_lines=False)
<ipython-input-144-fad40ba94602>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cx_liq_ativ_financ_tri['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri['DT_REFER'])


In [ ]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
cx_liq_ativ_financ_tri['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
cx_liq_ativ_financ_tri['Quarter'] = cx_liq_ativ_financ_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = cx_liq_ativ_financ_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para Q1, Q2, etc.
pivot.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Definir os trimestres Q2 e Q3 que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 estiver presente
if 'Q2' in trimestres_presentes:
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q2 and coluna_q1:
        pivot[coluna_q2[0]] = pivot[coluna_q2[0]] - pivot[coluna_q1[0]]

# Verificar e calcular Q3 - Q2 - Q1 se Q3 estiver presente
if 'Q3' in trimestres_presentes:
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col]

    if coluna_q3 and coluna_q2 and coluna_q1:
        pivot[coluna_q3[0]] = pivot[coluna_q3[0]] - pivot[coluna_q2[0]] - pivot[coluna_q1[0]]

cx_liq_ativ_financ_trim = pivot




Salvando relatórios no diretório

In [ ]:
# Diretório base onde os arquivos CSV serão salvos
diretorio_base = diretorio_rel_ind_financeiros

#Relatórios
receita_liquida.to_csv(os.path.join(diretorio_base, 'receita_liquida.csv'), sep=';', index=False)
receita_liquida_trim.to_csv(os.path.join(diretorio_base, 'receita_liquida_trim.csv'), sep=';', index=False)
ativo_total.to_csv(os.path.join(diretorio_base, 'ativo_total.csv'), sep=';', index=False)
ativo_total_trim.to_csv(os.path.join(diretorio_base, 'ativo_total_trim.csv'), sep=';', index=False)
patrimonio_liquido.to_csv(os.path.join(diretorio_base, 'patrimonio_liquido.csv'), sep=';', index=False)
patrimonio_liquido_trim.to_csv(os.path.join(diretorio_base, 'patrimonio_liquido_trim.csv'), sep=';', index=False)
lucro_liquido.to_csv(os.path.join(diretorio_base, 'lucro_liquido.csv'), sep=';', index=False)
lucro_liquido_trim.to_csv(os.path.join(diretorio_base, 'lucro_liquido_trim.csv'), sep=';', index=False)
divida_liquida.to_csv(os.path.join(diretorio_base, 'divida_liquida.csv'), sep=';', index=False)
divida_liquida_trim.to_csv(os.path.join(diretorio_base, 'divida_liquida_trim.csv'), sep=';', index=False)
EBTIDA.to_csv(os.path.join(diretorio_base, 'EBTIDA.csv'), sep=';', index=False)
EBTIDA_trim.to_csv(os.path.join(diretorio_base, 'EBTIDA_trim.csv'), sep=';', index=False)
free_cash_flow.to_csv(os.path.join(diretorio_base, 'free_cash_flow.csv'), sep=';', index=False)
free_cash_flow_trim.to_csv(os.path.join(diretorio_base, 'free_cash_flow_trim.csv'), sep=';', index=False)
cx_liq_ativ_financ.to_csv(os.path.join(diretorio_base, 'cx_liq_ativ_financ.csv'), sep=';', index=False)
cx_liq_ativ_financ_trim.to_csv(os.path.join(diretorio_base, 'cx_liq_ativ_financ_trim.csv'), sep=';', index=False)


Lucro por ação (DRE)

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.99')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
lucro_acao = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
lucro_acao.name = None


lucro_acao

Ano,DENOM_CIA,CNPJ_CIA,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,2W ECOBANK S.A.,08.773.135/0001-00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,10.345.009/0001-98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
675,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
676,ZAMP SA,13.574.594/0001-96,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
677,ÉVORA SA,91.820.068/0001-72,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vendas Líquidas

In [ ]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_ind"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.01.01')|(df['CD_CONTA']=='3.01.02')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
vendas_liquidas = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
vendas_liquidas.name = None


vendas_liquidas

Ano,DENOM_CIA,CNPJ_CIA,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ACRUX SECURITIZADORA S/A,07.825.881/0001-29,60060.0,84993.0,97947.0,33219.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALUPAR INVESTIMENTO S/A,08.364.948/0001-38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90318.0,NaN,NaN
2,AMPLA ENERGIA E SERVICOS S.A.,33.050.071/0001-58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ARGO TRANSMISSÃO DE ENERGIA S/A,24.624.490/0001-65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1274404.0,882365.0,752248.0
4,ARGO V TRANSMISSÃO DE ENERGIA S.A.,20.514.590/0001-88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95992.0,122988.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,VIX LOGÍSTICA S/A,32.681.371/0001-72,778135.0,913803.0,1046459.0,975106.0,856603.0,795772.0,900099.0,NaN,NaN,NaN,NaN
199,XX DE NOVEMBRO INVESTIMENTOS E PARTICIPAÇÕES S.A.,03.538.833/0001-07,23743.0,17169.0,20593.0,14093.0,11665.0,-451.0,0.0,0.0,10284.0,10931.0,9063.0
200,ÁGUAS DO SERTÃO S.A.,45.456.117/0001-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55782.0
201,ÁQUILLA SECURITIZADORA S.A.,09.439.783/0001-89,19123914.0,18453016.0,17265829.0,16799523.0,17394020.0,15604537.0,9915382.0,NaN,NaN,NaN,NaN


Número de ações

Obtenção dos relatórios dos formulários de referencia (FRE). Há uma url para os relatórios dos ultimos 5 anos e uma url para os relatórios mais antigos, por isso a necessidade de separa em 2 urls os requerimentos.

In [ ]:
# URL base dos relatórios até 2018
url_base_mais_5_anos = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/"

# URL base dos relatórios a partir de 2019
url_base_ultimos_5_anos = "https://dados.cvm.gov.br/dataset/cia_aberta-doc-fre"

# Período
ano_atual = datetime.now().year
rel_anteriores = range(ano_atual - 11, ano_atual - 4)
rel_atuais = range(ano_atual - 4, ano_atual  + 1)

# Diretório de destino para download
diretorio_download = '/content/drive/MyDrive/arquivos_cvm_zip/Formulários de referencia (FRE)/Download'

# Diretório de destino para descompactação
diretorio_descompactacao = '/content/drive/MyDrive/arquivos_cvm_zip/Formulários de referencia (FRE)'

# Criando os diretórios se não existirem
os.makedirs(diretorio_download, exist_ok=True)
os.makedirs(diretorio_descompactacao, exist_ok=True)

# Iterando relatórios com mais de 5 anos
for ano in rel_anteriores:
    # Construindo o caminho do arquivo local para download
    caminho_arquivo_zip = os.path.join(diretorio_download, f"FRE_CIA_ABERTA_{ano}.zip")

    # Verificando se o arquivo já existe
    if os.path.exists(caminho_arquivo_zip):
        print(f"O arquivo para {ano} já foi baixado. Pulando para o próximo ano.")
    else:
        # Construindo a URL para o ano específico
        url_ano = f"{url_base_mais_5_anos}FRE_CIA_ABERTA_{ano}.zip"

        # Obtendo o conteúdo da página
        response = requests.get(url_ano, stream=True)

        # Verificando se a requisição foi bem-sucedida
        if response.status_code == 200:
            # Baixando o arquivo com barra de progresso
            with open(caminho_arquivo_zip, 'wb') as file, tqdm(
                desc=f"Fazendo download de {ano}",
                total=int(response.headers.get('content-length', 0)),
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    bar.update(len(data))
                    file.write(data)

            print(f"Download concluído para {ano}.")
        else:
            print(f"Falha ao obter o arquivo para {ano}.")

    # Construindo os caminhos dos arquivos
    diretorio_ano_descompactado = os.path.join(diretorio_descompactacao, str(ano))

    # Verificando se o diretório de descompactação já existe
    if os.path.exists(diretorio_ano_descompactado):
        print(f"Os arquivos para o ano {ano} já foram descompactados. Pulando para o próximo ano.")
    else:
        # Criando o diretório para o ano
        os.makedirs(diretorio_ano_descompactado, exist_ok=True)

        # Descompactando o arquivo ZIP no diretório do ano
        with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
            zip_ref.extractall(diretorio_ano_descompactado)

        print(f"Descompactação concluída para {ano}.")

# Iterando sobre os anos a partir de 2019
for ano in rel_atuais:
    # Construindo o caminho do arquivo local para download
    caminho_arquivo_zip = os.path.join(diretorio_download, f"FRE_CIA_ABERTA_{ano}.zip")

    # Verificando se o arquivo já existe
    if os.path.exists(caminho_arquivo_zip):
        print(f"O arquivo para {ano} já foi baixado. Pulando para o próximo ano.")
    else:
        # Escolhendo a URL base correta com base no ano
        url_base = url_base_ultimos_5_anos if ano >= 2019 else url_base_mais_5_anos

        # Construindo a URL para o ano específico
        url_ano = f"{url_base}FRE_CIA_ABERTA_{ano}.zip"

        # Obtendo o conteúdo da página
        response = requests.get(url_ano, stream=True)

        # Verificando se a requisição foi bem-sucedida
        if response.status_code == 200:
            # Baixando o arquivo com barra de progresso
            with open(caminho_arquivo_zip, 'wb') as file, tqdm(
                desc=f"Fazendo download de {ano}",
                total=int(response.headers.get('content-length', 0)),
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    bar.update(len(data))
                    file.write(data)

            print(f"Download concluído para {ano}.")
        else:
            print(f"Falha ao obter o arquivo para {ano}.")

    # Construindo os caminhos dos arquivos
    diretorio_ano_descompactado = os.path.join(diretorio_descompactacao, str(ano))

    # Verificando se o diretório de descompactação já existe
    if os.path.exists(diretorio_ano_descompactado):
        print(f"Os arquivos para o ano {ano} já foram descompactados. Pulando para o próximo ano.")
    else:
        # Criando o diretório para o ano
        os.makedirs(diretorio_ano_descompactado, exist_ok=True)

        # Descompactando o arquivo ZIP no diretório do ano
        with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
            zip_ref.extractall(diretorio_ano_descompactado)

        print(f"Descompactação concluída para {ano}.")

print("Downloads e descompactações concluídos.")


O arquivo para 2012 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2012 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2013 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2013 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2014 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2014 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2015 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2015 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2016 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2016 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2017 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2017 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2018 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2018 já foram descompactados. Pulando para 

In [ ]:
# Lista com todos os arquivos no diretório disponíveis para visualização
diretorio = '/content/drive/MyDrive/arquivos_cvm_zip/Formulários de referencia (FRE)/2012'
arquivos_disponiveis = os.listdir(diretorio)

# Exibe a lista de arquivos disponíveis
for arquivo in arquivos_disponiveis:
    print(arquivo)

fre_cia_aberta_2012.csv
fre_cia_aberta_administrador_membro_conselho_fiscal_2012.csv
fre_cia_aberta_ativo_imobilizado_2012.csv
fre_cia_aberta_ativo_intangivel_2012.csv
fre_cia_aberta_auditor_2012.csv
fre_cia_aberta_auditor_responsavel_2012.csv
fre_cia_aberta_capital_social_2012.csv
fre_cia_aberta_capital_social_aumento_2012.csv
fre_cia_aberta_capital_social_aumento_classe_acao_2012.csv
fre_cia_aberta_capital_social_classe_acao_2012.csv
fre_cia_aberta_capital_social_desdobramento_2012.csv
fre_cia_aberta_capital_social_desdobramento_classe_acao_2012.csv
fre_cia_aberta_capital_social_reducao_2012.csv
fre_cia_aberta_capital_social_reducao_classe_acao_2012.csv
fre_cia_aberta_capital_social_titulo_conversivel_2012.csv
fre_cia_aberta_direito_acao_2012.csv
fre_cia_aberta_distribuicao_capital_2012.csv
fre_cia_aberta_distribuicao_capital_classe_acao_2012.csv
fre_cia_aberta_distribuicao_dividendos_2012.csv
fre_cia_aberta_distribuicao_dividendos_classe_acao_2012.csv
fre_cia_aberta_endividamento_20

In [ ]:
# Visualizando relatório a ser aberto
relatorio = "fre_cia_aberta_endividamento"  # Inserir apenas o nome do arquivo, sem o ano
ano = 2020

# Definindo o caminho do arquivo usando formatação de string
caminho_arquivo = f'/content/drive/MyDrive/arquivos_cvm_zip/Formulários de referencia (FRE)/{ano}/{relatorio}_{ano}.csv'

df = pd.read_csv(caminho_arquivo, encoding='cp1252', sep=';')
df

,CNPJ_Companhia,Data_Referencia,Versao,ID_Documento,Data_Inicio_Exercicio_Social,Data_Fim_Exercicio_Social,Passivo,Indice_Endividamento,Tipo_Indice,Descricao_Outro_Indice
0,00.000.000/0001-91,2020-01-01,28,104726,2019-01-01,2019-12-31,1.342295e+12,12.500000,Índice de Endividamento,NaN
1,00.000.208/0001-00,2020-01-01,14,103809,2019-01-01,2019-12-31,1.512080e+10,8.700000,Índice de Endividamento,NaN
2,00.001.180/0001-26,2020-01-01,22,104768,2019-01-01,2019-12-31,1.143829e+11,1.602132,Índice de Endividamento,NaN
3,00.070.698/0001-11,2020-01-01,7,103845,2019-01-01,2019-12-31,2.744266e+09,0.761715,Índice de Endividamento,NaN
4,00.192.769/0001-59,2020-01-01,1,94800,2019-01-01,2019-12-31,6.844000e+07,3.378606,Índice de Endividamento,NaN
...,...,...,...,...,...,...,...,...,...,...
793,96.418.264/0218-02,2020-01-01,10,103744,2020-01-01,2020-12-31,1.499136e+09,10.291599,Índice de Endividamento,NaN
794,96.418.264/0218-02,2020-01-01,10,103744,2020-01-01,2020-12-31,1.477699e+09,10.252685,Índice de Endividamento,NaN
795,97.191.902/0001-94,2020-01-01,1,95281,2019-01-01,2019-12-31,2.310906e+08,1.210000,Índice de Endividamento,NaN
796,97.837.181/0001-47,2020-01-01,13,104593,2019-01-01,2019-12-31,5.782521e+09,1.172410,Índice de Endividamento,NaN


Número de ações emitidas

In [ ]:
# Definindo relatório a ser aberto
anos = range(ano_inicial, ano_final+1)

# Criando um DataFrame vazio para armazenar os dados
df_numero_acoes = pd.DataFrame()

# Iterando pelos anos
for ano in anos:
    # Construindo o caminho do arquivo
    caminho_arquivo = f'/content/drive/MyDrive/arquivos_cvm_zip/Formulários de referencia (FRE)/{ano}/fre_cia_aberta_capital_social_{ano}.csv'

    # Verificando se o arquivo existe antes de tentar lê-lo
    if os.path.exists(caminho_arquivo):
        # Lendo o arquivo CSV
        df = pd.read_csv(caminho_arquivo, encoding='cp1252', sep=';')

        # Filtrando o DataFrame para as colunas desejadas e o Tipo_Capital correto
        df_filtrado = df[df['Tipo_Capital'] == 'Capital Emitido'][['CNPJ_Companhia', 'Quantidade_Total_Acoes']]

        # Agrupando por CNPJ e somando as quantidades de ações para cada CNPJ
        df_filtrado = df_filtrado.groupby('CNPJ_Companhia')['Quantidade_Total_Acoes'].sum().reset_index()

        # Renomeando a coluna Quantidade_Total_Acoes para o ano correspondente
        df_filtrado = df_filtrado.rename(columns={'Quantidade_Total_Acoes': str(ano)})

        # Adicionando os dados ao DataFrame final
        if df_numero_acoes.empty:
            df_numero_acoes = df_filtrado
        else:
            df_numero_acoes = pd.merge(df_numero_acoes, df_filtrado, on='CNPJ_Companhia', how='outer')

df_numero_acoes


,CNPJ_Companhia,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,00.000.000/0001-91,2.865417e+09,2.865417e+09,2.865417e+09,2.865417e+09,2.865417e+09,2.865417e+09,2.865417e+09,2.865417e+09,2.865417e+09,2.865417e+09,NaN
1,00.001.180/0001-26,1.352634e+09,1.352634e+09,1.352634e+09,1.352634e+09,1.352634e+09,1.352634e+09,1.352634e+09,2.921565e+09,1.568931e+09,1.568931e+09,6.804934e+09
2,00.108.786/0001-65,6.817027e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00.185.475/0001-08,1.226197e+07,1.226197e+07,1.671949e+07,1.671949e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00.249.786/0001-85,5.395243e+07,5.395242e+07,5.395242e+07,5.395242e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
878,62.070.362/0001-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.261701e+07
879,68.400.225/0001-79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.187428e+08
880,80.680.093/0001-81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.269683e+06
881,87.870.952/0001-44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.331640e+05
